<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Python - Type annotations

by [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/), [Marius van den Beek](https://training.galaxyproject.org/hall-of-fame/mvdbeek/), [Peter van Heusden](https://training.galaxyproject.org/hall-of-fame/pvanheus/), [Donny Vrins](https://training.galaxyproject.org/hall-of-fame/dirowa/), [Bazante Sanders](https://training.galaxyproject.org/hall-of-fame/bazante1/), [Avans Hogeschool](https://training.galaxyproject.org/hall-of-fame/avans-atgm/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- What is typing?
- How does it improve code?
- Can it help me?

**Objectives**

- Understand the utility of annotating types on one's code
- Understand the limits of type annotations in python

**Time Estimation: 30M**
</div>


<p>In some languages type annotations are a core part of the language and types are checked at compile time, to ensure your code can never use the incorrect type of object. Python, and a few other dynamic languages, instead use <a href="https://en.wikipedia.org/wiki/Duck_typing">“Duck Typing”</a> wherein the type of the object is less important than whether or not the correct methods or attributes are available.</p>
<p>However, we can provide type hints as we write python which will allow our editor to type check code as we go, even if it is not typically enforced at any point.</p>
<blockquote class="agenda" style="border: 2px solid #86D486;display: none; margin: 1em 0.2em">
<div class="box-title" aria-label="agenda box: Agenda" style="font-size: 150%"> Agenda</div>
<p>In this tutorial, we will cover:</p>
<ol id="markdown-toc">
<li><a href="#types" id="markdown-toc-types">Types</a></li>
</ol>
</blockquote>
<h2 id="types">Types</h2>
<p>Types used for annotations can be any of the base types:</p>


In [ ]:
str
int
float
bool
None
...

<p>or they can be relabeling of existing types, letting you create new types as needed to represent your internal data structures</p>


In [ ]:
from typing import NewType

NameType = NewType("NameType", str)
Point2D = NewType("Point2D", tuple[float, float])

<blockquote class="tip" style="border: 2px solid #FFE19E; margin: 1em 0.2em">
<div class="box-title" aria-label="tip box: Tip: Does <code style=&quot;color: inherit&quot;>tuple</code> cause an issue?" style="font-size: 150%">💡 Tip: Does <code style=&quot;color: inherit&quot;>tuple</code> cause an issue?</div>
<p>You might be on a python earlier than 3.9. Please update, or rewrite these as Tuple and List which must be imported.</p>
</blockquote>
<h2 id="but-why">But why?</h2>
<p>Imagine for a minute you have a situation like the following, take a minute to read and understand the code:</p>


In [ ]:
# Fetch the user and history list
(history_id, user_id) = GetUserAndCurrentHistory("hexylena")

# And make sure all of the permissions are correct
history = History.fetch(history_id)
history.share_with(user_id)
history.save()

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title" aria-label="question box: Question" style="font-size: 150%">❓ Question</div>
<ol>
<li>Can you be sure the <code style="color: inherit">history_id</code> and <code style="color: inherit">user_id</code> are in the correct order? It
seems like potentially not, given the ordering of “user” and “history” in the
function name, but without inspecting the definition of that function we
won’t know.</li>
<li>What happens if <code style="color: inherit">history_id</code> and <code style="color: inherit">user_id</code> are swapped?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title" aria-label="solution box: Solution" style="font-size: 150%">👁 Solution</div>
<ol>
<li>This is unanswerable without the code.</li>
<li>
<p>Depending on the magnitude of <code style="color: inherit">history_id</code> and <code style="color: inherit">user_id</code>, those may be within allowable ranges. Take for example</p>
<table>
<thead>
<tr>
<th>User</th>
<th>History Id</th>
</tr>
</thead>
<tbody>
<tr>
<td>1</td>
<td>1</td>
</tr>
<tr>
<td>1</td>
<td>2</td>
</tr>
<tr>
<td>2</td>
<td>3</td>
</tr>
<tr>
<td>2</td>
<td>4</td>
</tr>
</tbody>
</table>
<p>Given <code style="color: inherit">user_id=1</code> and <code style="color: inherit">history_id=2</code> we may intend that the second row in our tables, history #2 owned by user #1, is shared with that user, as they’re the owner. But if those are backwards, we’ll get a situation where history #1 is actually associated with user #1, but instead we’re sharing with user #2. We’ve created a situation where we’ve accidentally shared the wrong history with the wrong user! This could be a GDPR violation for our system and cause a lot of trouble.</p>
</li>
</ol>
</details>
</blockquote>
<p>However, if we have type definitions for the <code style="color: inherit">UserId</code> and <code style="color: inherit">HistoryId</code> that declare them as their own types:</p>


In [ ]:
from typing import NewType

UserId = NewType("UserId", int)
HistoryId = NewType("HistoryId", int)

<p>And then defined on our function, e.g.</p>


In [ ]:
def GetUserAndCurrentHistory(username: str) -> tuple[UserId, HistoryId]:
    x = UserId(1) # Pretend this is fetching from the database
    y = HistoryId(2) # Likewise
    return (x, y)

<p>we would be able to catch that, even if we call the variable <code style="color: inherit">user_id</code>, it will still be typed checked.</p>


In [ ]:
history_id: HistoryId
user_id: UserId

(user_id, history_id) = GetUserAndCurrentHistory("hexylena")
(history_id, user_id) = GetUserAndCurrentHistory("hexylena")

<p>If we’re using a code editor with typing hints, e.g. VSCode with PyLance, we’ll see something like:</p>
<p><img src="...
AAAASUVORK5CYII=
"" alt="Screenshot of VSCode showing the functions from above. The version with history_id first has a bright red line under the function call of GetUserAndCurrentHistory. A popup tab shown on hovering over the function name shows that Expression of type UserId cannot be assigned to declared type HistoryId. UserId is incompatible with HistoryId" /></p>
<p>Here we see that we’re not allowed to call this function this way, it’s simply impossible.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title" aria-label="question box: Question" style="font-size: 150%">❓ Question</div>
<p>What happens if you execute this code?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title" aria-label="solution box: Solution" style="font-size: 150%">👁 Solution</div>
<p>It executes happily. Types are <strong>not enforced at runtime</strong>. So this case where they’re both custom types around an integer, Python sees that it expects an int in both versions of the function call, and that works fine for it. That is why we are repeatedly calling them “type hints”, they’re hints to your editor to show suggestions and help catch bugs, but they’re not enforced.
If you modified the line <code style="color: inherit">y = HistoryId(2)</code> to be something like <code style="color: inherit">y = "test"</code>, the code will also execute fine. Python doesn’t care that there’s suddenly a string where you promised and asked for, an int. It simply does not matter.</p>
<p>However, types <em>are</em> checked when you do operations involving them. Trying to get the <code style="color: inherit">len()</code> of an integer? That will raise an <code style="color: inherit">TypeError</code>, as integers don’t support the <code style="color: inherit">len()</code> call.</p>
</details>
</blockquote>
<h2 id="typing-variables">Typing Variables</h2>
<p>Adding types to variables is easy, you’ve seen a few examples already:</p>


In [ ]:
a: str = "Hello"
b: int = 3
c: float = 3.14159
d: bool = True

<h3 id="complex-types">Complex Types</h3>
<p>But you can go further than this with things like <code style="color: inherit">tuple</code> and <code style="color: inherit">list</code> types:</p>


In [ ]:
e: list[int] = [1, 2, 3]
f: tuple[int, str] = (3, "Hi.")
g: list[tuple[int, int]] = [(1, 2), (3, 4)]

<h3 id="typing-functions">Typing Functions</h3>
<p>Likewise you’ve seen an example of adding type hints to a function:</p>


In [ ]:
def reverse_list_of_ints(a: list[int]) -> list[int]:
    return a[::-1]

<p>But this is a very specific function, right? We can reverse lists with more than just integers. For this, you can use <code style="color: inherit">Any</code>:</p>


In [ ]:
from typing import Any

def reverse_list(a: list[Any]) -> list[Any]:
    return a[::-1]

<p>But this will lose the type information from the start of the function to the end. You said it was a <code style="color: inherit">list[Any]</code> so your editor might not provide any type hints there, even though you could know, that calling it with a <code style="color: inherit">list[int]</code> would always return the same type. Instead you can do</p>


In [ ]:
from typing import TypeVar

T = TypeVar("T") # Implicitly any

def reverse_list(a: list[T]) -> list[T]:
    return a[::-1]

<p>Now this will allow the function to accept a list of any type of value, int, float, etc. But it will also accept types you might not have intended:</p>


In [ ]:
w: list[tuple[int, int]] = [(1, 2), (3, 4), (5, 8)]
reverse_list(w)

<p>We can lock down what types we’ll accept by using a <code style="color: inherit">Union</code> instead of <code style="color: inherit">Any</code>. With a <code style="color: inherit">Union</code>, we can define that a type in that position might be any one of a few more specific types. Say your function can only accept strings, integers, or floats:</p>


In [ ]:
from typing import Union

def reverse_list(a: list[Union[int, float, str]]) -> list[Union[int, float, str]]:
    return a[::-1]

<p>Here we have used a <code style="color: inherit">Union[A, B, ...]</code> to declare that it can only be one of these three types.</p>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title" aria-label="question box: Question" style="font-size: 150%">❓ Question</div>
<ol class="solution">
<li>
<p>Are both of these valid definitions?`</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">q1: list[Union[int, float, str]] = [1, 2, 3]
q2: list[Union[int, float, str]] = [1, 2.3214, "asdf"]
</code></pre></div>      </div>
</li>
<li>
<p>If that wasn’t what you expected, how would you define it so that it would be?</p>
<blockquote>
<div class="box-title" aria-label="solution box: Solution" style="font-size: 150%">👁 Solution</div>
<p>Yes, both are valid, but maybe you expected a homogeneous list. If you wanted that, you could instead do</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">q3: Union[list[int], list[float], list[str]] = [1, 2, 3]
q4: Union[list[int], list[float], list[str]] = [1, 2.3243, "asdf"] # Fails
</code></pre></div>        </div>
</blockquote>
</li>
</ol>
</blockquote>
<h3 id="optional">Optional</h3>
<p>Sometimes you have an argument to a function that is truly optional, maybe you have a different code path if it isn’t there, or you simply process things differently but still correctly. You can explicitly declare this by defining it as <code class="language-plaintext highlighter-rouge">Optional</code></p>


In [ ]:
from typing import Optional

def pretty(lines: list[str], padding: Optional[str] = None) -> None:
    for line in lines:
        if padding:
            print(f"{padding} {line}")
        else:
            print(line)


lines = ["hello", "world", "你好", "世界"]

# Without the optional argument
pretty(lines)
# And with the optional
pretty(lines, "★")

<p>While this superficially <em>looks</em> like a keyword argument with a default value, however it’s subtly different. Here an explicit value of None is allowed, and we still know that it will either be a string, or it will be None. Not something that was possible with just a keyword argument.</p>
<h2 id="testing-for-types">Testing for Types</h2>
<p>You can use <code style="color: inherit">mypy</code> to ensure that these type annotations are working in a project, this is a step you could add to your automated testing, if you have that. Using the <code style="color: inherit">HistoryId</code>/<code style="color: inherit">UserId</code> example from above, we can write that out into a script and test it out by running <code style="color: inherit">mypy</code> on that file:</p>
<div class="language-console highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="gp">&#36;</span><span class="w"> </span>mypy tmp.py
<span class="go">tmp.py:15: error: Incompatible types in assignment (expression has type "UserId", variable has type "HistoryId")
tmp.py:15: error: Incompatible types in assignment (expression has type "HistoryId", variable has type "UserId")
</span></code></pre></div></div>
<p>Here it reports the errors in the console, and you can use this to prevent bad code from being committed.</p>
<h2 id="exercise">Exercise</h2>
<p>Here is an example module that would be stored in <code class="language-plaintext highlighter-rouge">corp/__init__.py</code></p>


In [ ]:
def repeat(x, n):
    """Return a list containing n references to x."""
    return [x]*n


def print_capitalized(x):
    """Print x capitalized, and return x."""
    print(x.capitalize())
    return x


def concatenate(x, y) :
    """Add two strings together."""
    return x + y

<p>And here are some example invocations of that module, as found in <code class="language-plaintext highlighter-rouge">test.py</code></p>


In [ ]:
from corp import *

x = repeat("A", 3) # Should return ["A", "A", "A"]
y = print_capitalized("hElLo WorLd") # Should print Hello World
z = concatenate("Hi", "Bob") # HiBob

<blockquote class="hands_on" style="border: 2px solid #dfe5f9; margin: 1em 0.2em">
<div class="box-title" aria-label="hands-on box: Hands-on: Add type annotations" style="font-size: 150%"> Hands-on: Add type annotations</div>
<ol>
<li>Add type annotations to each of those functions AND the variables <code style="color: inherit">x</code>, <code style="color: inherit">y</code>, <code style="color: inherit">z</code></li>
<li>How did you know which types were appropriate?</li>
<li>Does <code style="color: inherit">mypy</code> approve of your annotations? (Run <code style="color: inherit">mypy test.py</code>, once you’ve written the above files out to their appropriate locations.)</li>
</ol>
</blockquote>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title" aria-label="solution box: Solution" style="font-size: 150%">👁 Solution</div>
<ol>
<li>
<p>The proper annotations:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">def repeat(x: str, n: int) -&gt; list[str]:
# Or
from typing import TypeVar
T = TypeVar("T")
def repeat(x: T, n: int) -&gt; list[T]:

def print_capitalized(x: str) -&gt; str:

def concatenate(x: str, y:str) -&gt; str:
</code></pre></div>      </div>
<p>and</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">x: list[str] = ...
y: str = ...
z: str = ...
</code></pre></div>      </div>
</li>
<li>You might have discovered this by a combination of looking at the function definitions and their documentation, and perhaps also the sample invocations and what types were passed there.</li>
<li>We hope so!</li>
</ol>
</details>
<h2 id="automation-with-monkeytype">Automation with MonkeyType</h2>
<p>You can use MonkeyType to automatically apply type annotations to your code. Based on the execution of the code, it will make a best guess about what types are supported.</p>
<blockquote class="hands_on" style="border: 2px solid #dfe5f9; margin: 1em 0.2em">
<div class="box-title" aria-label="hands-on box: Hands-on: Using MonkeyType to generate automatic annotations" style="font-size: 150%"> Hands-on: Using MonkeyType to generate automatic annotations</div>
<ol>
<li>Create a folder for a module named <code style="color: inherit">some</code></li>
<li>Touch <code style="color: inherit">some/__init__.py</code> to ensure it’s importable as a python module</li>
<li>
<p>Create <code style="color: inherit">some/module.py</code> and add the following contents:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">def add(a, b):
    return a + B
</code></pre></div>      </div>
</li>
<li>
<p>Create a script that uses that module:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">from some.module import add

add(1, 2)
</code></pre></div>      </div>
</li>
<li><code style="color: inherit">pip install monkeytype</code></li>
<li>
<p>Run MonkeyType to generate the annotations</p>
<div class="language-console highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="go">monkeytype run myscript.py
</span></code></pre></div>      </div>
</li>
<li>
<p>View the generated annotations</p>
<div class="language-console highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit"><span class="go">monkeytype stub myscript.py
</span></code></pre></div>      </div>
</li>
</ol>
</blockquote>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title" aria-label="question box: Question" style="font-size: 150%">❓ Question</div>
<ol>
<li>What was the output of that command?</li>
<li>This function will accept strings as well, add a statement to exercise that in <code style="color: inherit">myscript.py</code> and re-run <code style="color: inherit">monkeytype run</code> and <code style="color: inherit">monkeytype stub</code>. What is the new output?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title" aria-label="solution box: Solution" style="font-size: 150%">👁 Solution</div>
<ol>
<li>
<p>The expected output is:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">def add(a: int, b: int) -&gt; int: ...
</code></pre></div>        </div>
</li>
<li>
<p>You can add a statement like <code style="color: inherit">add("a", "b")</code> below <code style="color: inherit">add(1, 2)</code> to see:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">def add(a: Union[int, str], b: Union[int, str]) -&gt; Union[int, str]: ...
</code></pre></div>        </div>
</li>
</ol>
</blockquote>
</blockquote>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title" aria-label="question box: Question" style="font-size: 150%">❓ Question</div>
<p>Why is it different?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title" aria-label="solution box: Solution" style="font-size: 150%">👁 Solution</div>
<p>Because MonkeyType works by running the code provided (<code class="language-plaintext highlighter-rouge">myscript.py</code>) and annotating based on what executions it saw. In the first invocation it had not seen any calls to <code style="color: inherit">add()</code> with strings, so it only reported <code style="color: inherit">int</code> as acceptable types. However, the second time it saw <code style="color: inherit">str</code>s as well. Can you think of another type that would be supported by this operation, that was not caught? (list!)</p>
</blockquote>
</blockquote>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title" aria-label="question box: Question" style="font-size: 150%">❓ Question</div>
<ol>
<li>Does that type annotation make sense based on what you’ve learned today?</li>
<li>Can you write a better type annoation based on what you know?</li>
</ol>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title" aria-label="solution box: Solution" style="font-size: 150%">👁 Solution</div>
<ol>
<li>It works, but it’s not a great type annotation. Here the description looks like it can accept two <code style="color: inherit">int</code>s and return a <code style="color: inherit">str</code> which isn’t correct.</li>
<li>
<p>Here is a better type annotation</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">from typing import TypeVar
T = TypeVar("T", int, str, list)

def add(a: T, b: T) -&gt; T:
    return a + b
</code></pre></div>        </div>
</li>
</ol>
</blockquote>
</blockquote>


# Key Points

- Typing improves the correctness and quality of your code
- It can ensure that editor provided hints are better and more accurate.

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/data-science/tutorials/python-typing/tutorial.html#feedback) and check there for further resources!
